In [1]:
import cv2 as cv
import numpy as np
import os
from matplotlib import pyplot as plt
import time
import mediapipe as mp

In [2]:
mp_pose = mp.solutions.pose
mp_drawing = mp.solutions.drawing_utils

In [3]:
main_dir = os.getcwd()
main_dir

'C:\\Users\\nguye\\PycharmProjects\\Project_DPL302\\Pose recognition'

In [4]:
label = 'Yoga'
labels_path = os.path.join(main_dir,'dataset',label)
labels = os.listdir(labels_path)

In [5]:
poses = np.array(labels)
print(poses)

['downdog' 'plank' 'warrior2']


In [66]:
for pose_action in poses:
    try: 
        os.makedirs(os.path.join(main_dir, 'keypoints',pose_action))
    except:
        pass

In [67]:
img_no = 0
with mp_pose.Pose(static_image_mode = True, min_detection_confidence=0.5) as pose:
    for pose_actions in poses:
        current_pose = pose_actions
        pose_path = os.path.join(main_dir,'dataset', label, pose_actions)
        pose_actions = np.array(os.listdir(pose_path))
        img_no = 0
        for img in pose_actions:
            img_path = os.path.join(main_dir,'dataset', label, current_pose, img)
            frame = cv.imread(img_path)
            
            frame.flags.writeable = False
            frame = cv.cvtColor(frame, cv.COLOR_BGR2RGB)
            results = pose.process(frame)
            frame.flags.writeable = True
            frame = cv.cvtColor(frame, cv.COLOR_RGB2BGR)
            print('Extracting ', img, 'of', current_pose)
            
            keypoints = np.array([[result.x, result.y, result.z, result.visibility] for result in results.pose_landmarks.landmark]).flatten() if results.pose_landmarks else np.zeros(33*4) 
            
            mp_drawing.draw_landmarks(frame, results.pose_landmarks, mp_pose.POSE_CONNECTIONS)
            
            kp_path = os.path.join(main_dir, 'keypoints', current_pose, str(img_no))
            np.save(kp_path, keypoints)
            img_no += 1
                
            cv.imshow('Reading', frame)
            cv.waitKey(100)


cv.destroyAllWindows()

Extracting  00000000.jpg of downdog
Extracting  00000001.JPG of downdog
Extracting  00000002.jpg of downdog
Extracting  00000003.jpg of downdog
Extracting  00000004.jpg of downdog
Extracting  00000005.jpg of downdog
Extracting  00000006.jpg of downdog
Extracting  00000007.jpg of downdog
Extracting  00000008.jpg of downdog
Extracting  00000009.jpg of downdog
Extracting  00000011.jpg of downdog
Extracting  00000012.jpg of downdog
Extracting  00000013.jpg of downdog
Extracting  00000014.jpg of downdog
Extracting  00000016.jpg of downdog
Extracting  00000017.png of downdog
Extracting  00000018.jpg of downdog
Extracting  00000019.jpg of downdog
Extracting  00000020.jpg of downdog
Extracting  00000021.jpg of downdog
Extracting  00000022.jpg of downdog
Extracting  00000023.jpg of downdog
Extracting  00000024.jpg of downdog
Extracting  00000025.jpg of downdog
Extracting  00000026.jpg of downdog
Extracting  00000027.jpg of downdog
Extracting  00000029.jpg of downdog
Extracting  00000030.jpg of 

In [6]:
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical

In [7]:
label_map = {label:num for num, label in enumerate(poses)}

In [8]:
label_map

{'downdog': 0, 'plank': 1, 'warrior2': 2}

In [9]:
poses_kp, labels_kp = [], []
for pose_action in poses:
    pose_path = os.path.join(main_dir, 'keypoints', pose_action)
    pose_count = len(os.listdir(pose_path))
    for pose_kp in range(pose_count):
        keypoints = np.load(os.path.join(main_dir, 'keypoints', pose_action,'{}.npy'.format(pose_kp)))
        poses_kp.append(keypoints)
        labels_kp.append(label_map[pose_action])
    

In [10]:
np.array(poses_kp).shape

(1061, 132)

In [11]:
np.array(labels_kp).shape

(1061,)

In [12]:
X = np.array(poses_kp)
X.shape

(1061, 132)

In [13]:
y = to_categorical(labels_kp).astype(int)
y.shape

(1061, 3)

In [14]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2)

In [15]:
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers import Dropout
from keras.layers import Conv1D
from keras.layers import MaxPooling1D

In [16]:
model = Sequential()
model.add(Conv1D(filters = 64, kernel_size = 3, activation='relu',input_shape=(132,1)))
model.add(Conv1D(filters = 64, kernel_size = 3, activation='relu'))
model.add(MaxPooling1D())
model.add(Flatten())
model.add(Dense(100, activation= 'relu'))
model.add(Dense(3, activation= 'softmax'))

In [17]:
from tensorflow.keras.optimizers.legacy import Adam
opt = Adam(learning_rate=0.0001)

In [18]:
model.compile(loss='categorical_crossentropy', optimizer =opt, metrics = ['accuracy'])

In [19]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv1d (Conv1D)             (None, 130, 64)           256       
                                                                 
 conv1d_1 (Conv1D)           (None, 128, 64)           12352     
                                                                 
 max_pooling1d (MaxPooling1  (None, 64, 64)            0         
 D)                                                              
                                                                 
 flatten (Flatten)           (None, 4096)              0         
                                                                 
 dense (Dense)               (None, 100)               409700    
                                                                 
 dense_1 (Dense)             (None, 3)                 303       
                                                        

In [21]:
model.fit(X_train, y_train, epochs = 100, verbose = True)

Epoch 1/100
27/27 [==============================] - 1s 12ms/step - loss: 1.0220 - accuracy: 0.5660
Epoch 2/100
27/27 [==============================] - 0s 11ms/step - loss: 0.8554 - accuracy: 0.6816
Epoch 3/100
27/27 [==============================] - 0s 11ms/step - loss: 0.6762 - accuracy: 0.7901
Epoch 4/100
27/27 [==============================] - 0s 11ms/step - loss: 0.5331 - accuracy: 0.8597
Epoch 5/100
27/27 [==============================] - 0s 11ms/step - loss: 0.4358 - accuracy: 0.8986
Epoch 6/100
27/27 [==============================] - 0s 11ms/step - loss: 0.3579 - accuracy: 0.9281
Epoch 7/100
27/27 [==============================] - 0s 12ms/step - loss: 0.3022 - accuracy: 0.9399
Epoch 8/100
27/27 [==============================] - 0s 11ms/step - loss: 0.2693 - accuracy: 0.9410
Epoch 9/100
27/27 [==============================] - 0s 12ms/step - loss: 0.2421 - accuracy: 0.9446
Epoch 10/100
27/27 [==============================] - 0s 12ms/step - loss: 0.2201 - accuracy: 0.9517

In [22]:
model.save('model.h5')

c:\users\nguye\pycharmprojects\project_dpl302\venv\lib\site-packages\keras\src\engine\training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [20]:
model.load_weights('model.h5')

In [21]:
pred = model.predict(X_test)
y_pred = pred.argmax(axis = -1).tolist()
print(y_pred)

7/7 [==============================] - 0s 4ms/step
[0, 0, 2, 1, 2, 0, 1, 2, 0, 0, 0, 0, 0, 2, 2, 0, 1, 0, 0, 0, 0, 1, 1, 2, 1, 1, 2, 2, 2, 1, 1, 0, 1, 1, 2, 0, 1, 0, 1, 2, 2, 1, 2, 0, 1, 1, 1, 1, 1, 0, 1, 1, 0, 0, 1, 1, 1, 0, 0, 2, 1, 2, 2, 1, 0, 0, 2, 1, 1, 1, 2, 2, 2, 2, 2, 2, 1, 1, 2, 2, 1, 2, 0, 0, 1, 2, 2, 0, 1, 1, 1, 1, 2, 2, 0, 2, 1, 2, 1, 1, 2, 1, 0, 1, 1, 1, 1, 2, 1, 1, 2, 2, 2, 0, 2, 1, 2, 1, 2, 0, 1, 1, 0, 1, 0, 0, 0, 2, 2, 1, 1, 1, 0, 1, 1, 2, 2, 2, 1, 1, 2, 1, 2, 1, 1, 2, 1, 1, 2, 0, 2, 2, 0, 2, 1, 0, 0, 2, 0, 2, 0, 1, 0, 0, 2, 1, 2, 1, 2, 0, 2, 1, 0, 0, 1, 1, 2, 2, 1, 0, 1, 0, 1, 1, 0, 0, 2, 0, 1, 2, 1, 0, 1, 0, 2, 1, 2, 0, 1, 1, 1, 1, 1, 2, 1, 0, 2, 1, 0, 0, 1, 2, 1]


In [22]:
y_test = y_test.argmax(axis = -1).tolist()
print(y_test)

[0, 0, 2, 1, 2, 0, 1, 2, 0, 0, 0, 0, 0, 2, 2, 0, 1, 0, 0, 0, 0, 1, 1, 2, 1, 1, 2, 2, 2, 1, 1, 0, 1, 1, 2, 0, 1, 0, 1, 2, 2, 1, 2, 0, 1, 1, 1, 1, 1, 0, 1, 1, 0, 0, 1, 1, 1, 0, 0, 2, 1, 2, 2, 1, 0, 0, 2, 1, 1, 1, 2, 2, 2, 2, 2, 2, 1, 1, 2, 2, 1, 2, 0, 0, 1, 2, 2, 0, 1, 1, 1, 1, 2, 2, 0, 2, 1, 2, 1, 1, 2, 1, 0, 1, 1, 1, 1, 2, 1, 1, 2, 2, 2, 0, 2, 1, 2, 1, 2, 0, 1, 1, 0, 1, 0, 0, 0, 2, 2, 1, 1, 1, 0, 1, 1, 2, 2, 2, 1, 1, 2, 1, 2, 1, 1, 2, 1, 1, 2, 0, 2, 2, 0, 2, 1, 0, 0, 2, 0, 2, 0, 1, 0, 0, 2, 1, 2, 1, 2, 0, 2, 1, 0, 0, 1, 1, 2, 2, 1, 0, 1, 0, 1, 1, 0, 0, 2, 0, 1, 2, 1, 0, 1, 0, 2, 1, 2, 1, 1, 1, 1, 1, 1, 2, 1, 0, 2, 1, 0, 0, 1, 2, 1]


In [23]:
from sklearn.metrics import accuracy_score

In [24]:
accuracy_score(y_test, y_pred)

0.9953051643192489

IMAGE TEST

In [25]:
image = "C:/Users/nguye/Downloads/downdog.jpg"
threshold = 0.7
with mp_pose.Pose(static_image_mode = True, min_detection_confidence=0.5) as pose:
    while True:
        cv.namedWindow("Result", cv.WINDOW_NORMAL)
        frame = cv.imread(image)

        frame.flags.writeable = False
        frame = cv.cvtColor(frame, cv.COLOR_BGR2RGB)
        results = pose.process(frame)
        frame.flags.writeable = True
        frame = cv.cvtColor(frame, cv.COLOR_RGB2BGR)

        keypoints = np.array([[result.x, result.y, result.z, result.visibility] for result in results.pose_landmarks.landmark]).flatten() if results.pose_landmarks else np.zeros(33*4)

        landmarks = results.pose_landmarks.landmark
        mp_drawing.draw_landmarks(frame, results.pose_landmarks, mp_pose.POSE_CONNECTIONS)
        
        pred = model.predict(np.expand_dims(keypoints, axis = 0))[0]
        y_pred = pred.argmax(axis=-1)        
        print(y_pred)

        if pred[np.argmax(pred)] > threshold:
            prediction = labels[np.argmax(pred)]

        cv.rectangle(frame, (0,0),(120,40),(245,117,16), -1)
        cv.putText(frame, prediction, (3,30), cv.FONT_HERSHEY_SIMPLEX, 1, (255,255,255), 2, cv.LINE_AA)
        cv.resizeWindow('Result', 800, 600)
        cv.imshow('Result',frame)
        
        if cv.waitKey(5) & 0xFF == ord('q'):
            break

cv.destroyAllWindows()



1/1 [==============================] - 0s 45ms/step
0
1/1 [==============================] - 0s 55ms/step
0
1/1 [==============================] - 0s 23ms/step
0
1/1 [==============================] - 0s 24ms/step
0
1/1 [==============================] - 0s 23ms/step
0
1/1 [==============================] - 0s 44ms/step
0
1/1 [==============================] - 0s 23ms/step
0
1/1 [==============================] - 0s 43ms/step
0
1/1 [==============================] - 0s 23ms/step
0
1/1 [==============================] - 0s 23ms/step
0
1/1 [==============================] - 0s 52ms/step
0
1/1 [==============================] - 0s 45ms/step
0
1/1 [==============================] - 0s 23ms/step
0
1/1 [==============================] - 0s 37ms/step
0
1/1 [==============================] - 0s 48ms/step
0
1/1 [==============================] - 0s 22ms/step
0
1/1 [==============================] - 0s 23ms/step
0
1/1 [==============================] - 0s 28ms/step
0
1/1 [=======================

VIDEO TEST

In [61]:
cap = cv.VideoCapture('C:/Users/nguye/Downloads/test1.mp4')
threshold = 0.95
sentence = []
with mp_pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.5) as pose:
    while cap.isOpened():
        ret, frame = cap.read()

        frame.flags.writeable = False
        frame = cv.cvtColor(frame, cv.COLOR_BGR2RGB) 
        results = pose.process(frame)
        frame.flags.writeable = True
        frame = cv.cvtColor(frame, cv.COLOR_RGB2BGR)

        keypoints = np.array([[result.x, result.y, result.z, result.visibility] for result in results.pose_landmarks.landmark]).flatten() if results.pose_landmarks else np.zeros(33*4)
        
        mp_drawing.draw_landmarks(frame, results.pose_landmarks, mp_pose.POSE_CONNECTIONS)

        pred = model.predict(np.expand_dims(keypoints, axis = 0))[0]
        y_pred = pred.argmax(axis=-1)
        print(y_pred)

        if pred[np.argmax(pred)] > threshold:
            sentence[:] = []
            sentence.append(labels[np.argmax(pred)])
        else:
            sentence[:] = []
            sentence.append(str('Unknown'))
    
        cv.rectangle(frame, (0,0),(120,40),(245,117,16), -1)
        cv.putText(frame, sentence[-1], (3,30), cv.FONT_HERSHEY_SIMPLEX, 1, (255,255,255), 2, cv.LINE_AA)
        
        cv.imshow('Result', frame)

        if cv.waitKey(10) & 0xFF == ord('q'):
            break
            
    cap.release()
    cv.destroyAllWindows()

LIVE TEST

In [57]:
cap = cv.VideoCapture(0)
threshold = 0.95
sentence = []
with mp_pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.5) as pose:
    while cap.isOpened():
        ret, frame = cap.read()

        frame.flags.writeable = False
        frame = cv.cvtColor(frame, cv.COLOR_BGR2RGB) 
        results = pose.process(frame)
        frame.flags.writeable = True
        frame = cv.cvtColor(frame, cv.COLOR_RGB2BGR)

        keypoints = np.array([[result.x, result.y, result.z, result.visibility] for result in results.pose_landmarks.landmark]).flatten() if results.pose_landmarks else np.zeros(33*4)
        
        mp_drawing.draw_landmarks(frame, results.pose_landmarks, mp_pose.POSE_CONNECTIONS)

        pred = model.predict(np.expand_dims(keypoints, axis = 0))[0]
        y_pred = pred.argmax(axis=-1)
        print(y_pred)

        if pred[np.argmax(pred)] > threshold:
            sentence[:] = []
            sentence.append(labels[np.argmax(pred)])
        else:
            sentence[:] = []
            sentence.append(str('Unknown'))
    
        cv.rectangle(frame, (0,0),(140,40),(245,117,16), -1)
        cv.putText(frame, sentence[-1], (3,30), cv.FONT_HERSHEY_SIMPLEX, 1, (255,255,255), 2, cv.LINE_AA)
        
        cv.imshow('Result', frame)

        if cv.waitKey(10) & 0xFF == ord('q'):
            break
            
    cap.release()
    cv.destroyAllWindows()

1/1 [==============================] - 0s 23ms/step
0
1/1 [==============================] - 0s 21ms/step
0
1/1 [==============================] - 0s 22ms/step
0
1/1 [==============================] - 0s 22ms/step
0
1/1 [==============================] - 0s 21ms/step
0
1/1 [==============================] - 0s 22ms/step
0
1/1 [==============================] - 0s 21ms/step
0
1/1 [==============================] - 0s 23ms/step
0
1/1 [==============================] - 0s 21ms/step
0
1/1 [==============================] - 0s 23ms/step
0
1/1 [==============================] - 0s 20ms/step
0
1/1 [==============================] - 0s 22ms/step
0
1/1 [==============================] - 0s 26ms/step
0
1/1 [==============================] - 0s 21ms/step
0
1/1 [==============================] - 0s 20ms/step
0
1/1 [==============================] - 0s 26ms/step
0
1/1 [==============================] - 0s 24ms/step
0
1/1 [==============================] - 0s 21ms/step
0
1/1 [=======================

ERROR POINTER

In [26]:
def nose():
    return [landmarks[mp_pose.PoseLandmark.NOSE.value].x, landmarks[mp_pose.PoseLandmark.NOSE.value].y]
def left_eye_inner():
    return [landmarks[mp_pose.PoseLandmark.LEFT_EYE_INNER.value].x, landmarks[mp_pose.PoseLandmark.LEFT_EYE_INNER.value].y]
def left_eye():
    return [landmarks[mp_pose.PoseLandmark.LEFT_EYE.value].x, landmarks[mp_pose.PoseLandmark.LEFT_EYE.value].y]
def left_eye_outer():
    return [landmarks[mp_pose.PoseLandmark.LEFT_EYE_OUTER.value].x, landmarks[mp_pose.PoseLandmark.LEFT_EYE_OUTER.value].y]
def right_eye_inner():
    return [landmarks[mp_pose.PoseLandmark.RIGHT_EYE_INNER.value].x, landmarks[mp_pose.PoseLandmark.RIGHT_EYE_INNER.value].y]
def right_eye():
    return [landmarks[mp_pose.PoseLandmark.RIGHT_EYE.value].x, landmarks[mp_pose.PoseLandmark.RIGHT_EYE.value].y]
def right_eye_outer():
    return [landmarks[mp_pose.PoseLandmark.RIGHT_EYE_OUTER.value].x, landmarks[mp_pose.PoseLandmark.RIGHT_EYE_OUTER.value].y]
def left_ear():
    return [landmarks[mp_pose.PoseLandmark.LEFT_EAR.value].x, landmarks[mp_pose.PoseLandmark.LEFT_EAR.value].y]
def right_ear():
    return [landmarks[mp_pose.PoseLandmark.RIGHT_EAR.value].x, landmarks[mp_pose.PoseLandmark.RIGHT_EAR.value].y]
def mouth_left():
    return [landmarks[mp_pose.PoseLandmark.MOUTH_LEFT.value].x, landmarks[mp_pose.PoseLandmark.MOUTH_LEFT.value].y]
def mouth_right():
    return [landmarks[mp_pose.PoseLandmark.MOUTH_RIGHT.value].x, landmarks[mp_pose.PoseLandmark.MOUTH_RIGHT.value].y]
def left_shoulder():
    return [landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].x, landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].y]
def right_shoulder():
    return [landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].x, landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].y]
def left_elbow():
    return [landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].x, landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].y]
def right_elbow():
    return [landmarks[mp_pose.PoseLandmark.RIGHT_ELBOW.value].x, landmarks[mp_pose.PoseLandmark.RIGHT_ELBOW.value].y]
def left_wrist():
    return [landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value].x, landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value].y]
def right_wrist():
    return [landmarks[mp_pose.PoseLandmark.RIGHT_WRIST.value].x, landmarks[mp_pose.PoseLandmark.RIGHT_WRIST.value].y]
def left_pinky():
    return [landmarks[mp_pose.PoseLandmark.LEFT_PINKY.value].x, landmarks[mp_pose.PoseLandmark.LEFT_PINKY.value].y]
def right_pinky():
    return [landmarks[mp_pose.PoseLandmark.RIGHT_PINKY.value].x, landmarks[mp_pose.PoseLandmark.RIGHT_PINKY.value].y]
def left_index():
    return [landmarks[mp_pose.PoseLandmark.LEFT_INDEX.value].x, landmarks[mp_pose.PoseLandmark.LEFT_INDEX.value].y]
def right_index():
    return [landmarks[mp_pose.PoseLandmark.RIGHT_INDEX.value].x, landmarks[mp_pose.PoseLandmark.RIGHT_INDEX.value].y]
def left_thumb():
    return [landmarks[mp_pose.PoseLandmark.LEFT_THUMB.value].x, landmarks[mp_pose.PoseLandmark.LEFT_THUMB.value].y]
def right_thumb():
    return [landmarks[mp_pose.PoseLandmark.RIGHT_THUMB.value].x, landmarks[mp_pose.PoseLandmark.RIGHT_THUMB.value].y]
def left_hip():
    return [landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].x, landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].y]
def right_hip():
    return [landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value].x, landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value].y]
def left_knee():
    return [landmarks[mp_pose.PoseLandmark.LEFT_KNEE.value].x, landmarks[mp_pose.PoseLandmark.LEFT_KNEE.value].y]
def right_knee():
    return [landmarks[mp_pose.PoseLandmark.RIGHT_KNEE.value].x, landmarks[mp_pose.PoseLandmark.RIGHT_KNEE.value].y]
def left_ankle():
    return [landmarks[mp_pose.PoseLandmark.LEFT_ANKLE.value].x, landmarks[mp_pose.PoseLandmark.LEFT_ANKLE.value].y]
def right_ankle():
    return [landmarks[mp_pose.PoseLandmark.RIGHT_ANKLE.value].x, landmarks[mp_pose.PoseLandmark.RIGHT_ANKLE.value].y]
def left_heel():
    return [landmarks[mp_pose.PoseLandmark.LEFT_HEEL.value].x, landmarks[mp_pose.PoseLandmark.LEFT_HEEL.value].y]
def right_heel():
    return [landmarks[mp_pose.PoseLandmark.RIGHT_HEEL.value].x, landmarks[mp_pose.PoseLandmark.RIGHT_HEEL.value].y]
def left_foot_index():
    return [landmarks[mp_pose.PoseLandmark.LEFT_FOOT_INDEX.value].x, landmarks[mp_pose.PoseLandmark.LEFT_FOOT_INDEX.value].y]
def right_foot_index():
    return [landmarks[mp_pose.PoseLandmark.RIGHT_FOOT_INDEX.value].x, landmarks[mp_pose.PoseLandmark.RIGHT_FOOT_INDEX.value].y]


In [27]:
kp_map = {
    0:nose,
    1:left_eye_inner,
    2:left_eye,
    3:left_eye_outer,
    4:right_eye_inner,
    5:right_eye,
    6:right_eye_outer,
    7:left_ear,
    8:right_ear,
    9:mouth_left,
    10:mouth_right,
    11:left_shoulder,
    12:right_shoulder,
    13:left_elbow,
    14:right_elbow,
    15:left_wrist,
    16:right_wrist,
    17:left_pinky,
    18:right_pinky,
    19:left_index,
    20:right_index,
    21:left_thumb,
    22:right_thumb,
    23:left_hip,
    24:right_hip,
    25:left_knee,
    26:right_knee,
    27:left_ankle,
    28:right_ankle,
    29:left_heel,
    30:right_heel,
    31:left_foot_index,
    32:right_foot_index
}

In [28]:
def get_3_cords(a, b, c):
    cord_1 = kp_map.get(a)()
    cord_2 = kp_map.get(b)()
    cord_3 = kp_map.get(c)()
    return cord_1, cord_2, cord_3

In [29]:
def get_angle(a,b,c):
    a = np.array(a)
    b = np.array(b)
    c = np.array(c)

    radians = np.arctan2(c[1]-b[1], c[0]-b[0]) - np.arctan2(a[1]-b[1], a[0]-b[0])
    angle = np.abs(radians*180.0/np.pi)

    if angle > 180.0:
        angle = 360 - angle

    return angle

In [30]:
def GetMainAngles():
    a, b, c = get_3_cords(13, 11, 23)
    up_left_1 = get_angle(a,b,c)

    a, b, c = get_3_cords(15, 13, 11)
    up_left_2 = get_angle(a,b,c)

    a, b, c = get_3_cords(14, 12, 24)
    up_right_1 = get_angle(a,b,c)

    a, b, c = get_3_cords(16, 14, 12)
    up_right_2 = get_angle(a,b,c)

    a, b, c = get_3_cords(11, 23, 25)
    down_left_1 = get_angle(a,b,c)

    a, b, c = get_3_cords(23, 25, 27)
    down_left_2 = get_angle(a,b,c)

    a, b, c = get_3_cords(12, 24, 26)
    down_right_1 = get_angle(a,b,c)

    a, b, c = get_3_cords(24, 26, 28)
    down_right_2 = get_angle(a,b,c)

    return up_left_1, up_left_2, up_right_1, up_right_2, down_left_1, down_left_2, down_right_1, down_right_2

In [37]:
def DrawError(ul1, ul2, ur1, ur2, dl1, dl2, dr1, dr2):
    #L-upper 1
    if  (ul1 < tul1 - range) or (ul1 > tul1 + range):
        cv.putText(frame, 'Wrong!!!',
                  tuple(np.multiply(left_shoulder(),[640,480]).astype(int)),
                       cv.FONT_HERSHEY_SIMPLEX,0.5, (0,0,255),2, cv.LINE_AA
                  )
    
    
    #L- upper 2
    if  (ul2 < tul2 - range) or (ul2 > tul2 + range):
        cv.putText(frame, 'Wrong!!!',
                  tuple(np.multiply(left_elbow(),[640,480]).astype(int)),
                       cv.FONT_HERSHEY_SIMPLEX,0.5, (0,0,255),2, cv.LINE_AA
                  )
        
    #R-upper 1
    if  (ur1 < tur1 - range) or (ur1 > tur1 + range):
        cv.putText(frame, 'Wrong!!!',
                  tuple(np.multiply(right_shoulder(),[640,480]).astype(int)),
                       cv.FONT_HERSHEY_SIMPLEX,0.5, (0,0,255),2, cv.LINE_AA
                  )
    
    
    #R-upper 2
    if  (ur2 < tur2 - range) or (ur2 > tur2 + range):
        cv.putText(frame, 'Wrong!!!',
                  tuple(np.multiply(right_elbow(),[640,480]).astype(int)),
                       cv.FONT_HERSHEY_SIMPLEX,0.5, (0,0,255),2, cv.LINE_AA
                  )
    
    #L-hip
    if  (dl1 < tdl1 - range) or (dl1 > tdl1 + range):
        cv.putText(frame, 'Wrong!!!',
                  tuple(np.multiply(left_hip(),[640,480]).astype(int)),
                       cv.FONT_HERSHEY_SIMPLEX,0.5, (0,0,255),2, cv.LINE_AA
                  )
    
    #L-Knee
    if  (dl2 < tdl2 - range) or (dl2 > tdl2 + range):
        cv.putText(frame, 'Wrong!!!',
                  tuple(np.multiply(left_knee(),[640,480]).astype(int)),
                       cv.FONT_HERSHEY_SIMPLEX,0.5, (0,0,255),2, cv.LINE_AA
                  )
    
    #R-hip 
    if  (dr1 < tdr1 - range) or (dr1 > tdr1 + range):
        cv.putText(frame, 'Wrong!!!',
                  tuple(np.multiply(right_hip(),[640,480]).astype(int)),
                       cv.FONT_HERSHEY_SIMPLEX,0.5, (0,0,255),2, cv.LINE_AA
                  )
    #R-Knee
    if  (dr2 < tdr2 - range) or (dr2 > tdr2 + range):
        cv.putText(frame, 'Wrong!!!',
                  tuple(np.multiply(right_knee(),[640,480]).astype(int)),
                       cv.FONT_HERSHEY_SIMPLEX,0.5, (0,0,255),2, cv.LINE_AA
                  )

In [38]:
def Downdog_check():
    global tul1, tul2, tur1, tur2, tdl1, tdl2, tdr1, tdr2
    tul1, tul2, tur1, tur2, tdl1, tdl2, tdr1, tdr2 = (175, 175, 175, 175, 60, 175, 60, 175)
    ul1, ul2, ur1, ur2, dl1, dl2, dr1, dr2 = GetMainAngles()
    
    DrawError(ul1, ul2, ur1, ur2, dl1, dl2, dr1, dr2)

In [39]:
def Plank_check():
    global tul1, tul2, tur1, tur2, tdl1, tdl2, tdr1, tdr2
    tul1, tul2, tur1, tur2, tdl1, tdl2, tdr1, tdr2 = (70, 178, 70, 178, 178, 178, 178, 178)
    ul1, ul2, ur1, ur2, dl1, dl2, dr1, dr2 = GetMainAngles()
    
    DrawError(ul1, ul2, ur1, ur2, dl1, dl2, dr1, dr2)

In [40]:
def Warrior_check():
    global tul1, tul2, tur1, tur2, tdl1, tdl2, tdr1, tdr2
    tul1, tul2, tur1, tur2, tdl1, tdl2, tdr1, tdr2 = (85, 175, 85, 175, 140, 180, 110, 107)
    ul1, ul2, ur1, ur2, dl1, dl2, dr1, dr2 = GetMainAngles()
     
    DrawError(ul1, ul2, ur1, ur2, dl1, dl2, dr1, dr2)

LIVE TEST W/ ERROR POINTER

In [41]:
cap = cv.VideoCapture(0)
threshold = 0.92
range = 15
sentence = []
with mp_pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.5) as pose:
    while cap.isOpened():
        ret, frame = cap.read()
        frame = cv.flip(frame, 1)

        frame.flags.writeable = False
        frame = cv.cvtColor(frame, cv.COLOR_BGR2RGB) 
        results = pose.process(frame)
        frame.flags.writeable = True
        frame = cv.cvtColor(frame, cv.COLOR_RGB2BGR)

        try: 
            landmarks = results.pose_landmarks.landmark
        except:
            pass
        
        keypoints = np.array([[result.x, result.y, result.z, result.visibility] for result in results.pose_landmarks.landmark]).flatten() if results.pose_landmarks else np.zeros(33*4)
        
        mp_drawing.draw_landmarks(frame, results.pose_landmarks, mp_pose.POSE_CONNECTIONS)

        pred = model.predict(np.expand_dims(keypoints, axis = 0))[0]
        y_pred = pred.argmax(axis=-1)


        if pred[np.argmax(pred)] > threshold and y_pred == 2:
            Warrior_check()

        if pred[np.argmax(pred)] > threshold and y_pred == 1:
            Plank_check()

        if pred[np.argmax(pred)] > threshold and y_pred == 0:
            Downdog_check()


        if pred[np.argmax(pred)] > threshold:
            sentence[:] = []
            sentence.append(labels[np.argmax(pred)])
        else:
            sentence[:] = []
            sentence.append(str('Unknown'))
    
        cv.rectangle(frame, (0,0),(140,40),(245,117,16), -1)
        cv.putText(frame, sentence[-1], (3,30), cv.FONT_HERSHEY_SIMPLEX, 1, (255,255,255), 2, cv.LINE_AA)
        
        cv.imshow('Result', frame)

        if cv.waitKey(10) & 0xFF == ord('q'):
            break
            
    cap.release()
    cv.destroyAllWindows()

1/1 [==============================] - 0s 28ms/step
